# Test with other healthy patients

first part: September 17, 2024

Addition made on January 7, 2026

AI was used for some technical parts

In [ ]:
import os
import pandas as pd
import nibabel as nib
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

from nilearn import datasets, input_data, connectome

In [ ]:
# Load the Oxford atlas
oxford = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')

# Access the labels and coordinates
atlas_labels = oxford['labels']
atlas_coords = oxford['maps']

In [ ]:
def process_patient(patient_id, dicom_root, output_root):
    print(f"Processing patient {patient_id}")

    # -------------------------
    # Paths
    # -------------------------
    dicom_input = os.path.join(dicom_root, patient_id)

    patient_out = os.path.join(output_root, patient_id)
    dicom_out = os.path.join(patient_out, f"{patient_id}_DICOM")
    nifti_out = os.path.join(patient_out, f"{patient_id}_nifti")

    os.makedirs(dicom_out, exist_ok=True)
    os.makedirs(nifti_out, exist_ok=True)

    nifti_file = os.path.join(nifti_out, f"{patient_id}_fmri.nii.gz")
    time_series_csv = os.path.join(patient_out, f"{patient_id}_time_series.csv")

    # -------------------------
    # DICOM → NIfTI
    # -------------------------
    settings.disable_validate_slice_increment()

    dicom2nifti.convert_directory(
        dicom_input,
        nifti_out,
        compression=True,
        reorient=True
    )

    # Find generated NIfTI
    nifti_file = [
        os.path.join(nifti_out, f)
        for f in os.listdir(nifti_out)
        if f.endswith(".nii") or f.endswith(".nii.gz")
    ][0]

    # -------------------------
    # Load atlas
    # -------------------------
    atlas = datasets.fetch_atlas_harvard_oxford(
        'cort-maxprob-thr25-2mm'
    )

    masker = input_data.NiftiLabelsMasker(
        labels_img=atlas.maps,
        standardize=True,
        memory='nilearn_cache'
    )

    # -------------------------
    # Extract time series
    # -------------------------
    fmri_img = nib.load(nifti_file)
    time_series = masker.fit_transform(fmri_img)

    # -------------------------
    # Save time series
    # -------------------------
    #df = pd.DataFrame(time_series, columns=atlas.labels)
    
    
    roi_labels = atlas.labels[1:]  # drop 'Background'
    df = pd.DataFrame(time_series, columns=roi_labels)
    df.to_csv(time_series_csv, index=False)

    print(f"Saved: {time_series_csv}")

    return time_series


In [ ]:
# set of patients added later

dicom_root = "/Users/mariamannone/Desktop/new_phd_/additional_patients_peppino_article/raw_dicom"
output_root = "/Users/mariamannone/Desktop/new_phd_/additional_patients_peppino_article/processed"

patients = [
    # CN = normal control
    
    # Group 1: more than 70 y.o.

    "002_S_0413", # F, 81, CN
    "002_S_0685", # F, 94, CN
    "002_S_1261", # F, 76, CN
    "002_S_1280", # F, 75, CN
    "002_S_0295", # M, 90, CN
    "006_S_0731", # M, 78, CN
 
    "018_S_5074", # F, 74, AD
    "019_S_4252", # F, 86, AD 
    "130_S_4589", # F, 75, AD
    "130_S_4641", # F, 74, AD
    "130_S_4660", # F, 77, AD
    "006_S_4192", # M, 82, AD
    "006_S_4546", # M, 71, AD
    #"006_S_4867", # M, 74, AD ## missing a ROI # removed
    "019_S_5012", # M, 76, AD
    "100_S_5106", # M, 74, AD
    
    # Group 2: less than 70 y.o.
    
    "006_S_4449", # F, 67, CN
    "012_S_4545", # F, 67, CN
    "012_S_4643", # F, 65, CN
    "002_S_5178", # M, 68, CN
    "031_S_4021", # M, 66, CN
    "100_S_4469", # M, 66, CN
    
    "130_S_4982", # F, 58, AD
    "031_S_4024", # F, 56, AD
    "018_S_5240", # F, 62, AD
    "130_S_4997", # F, 60, AD
    "010_S_5163", # M, 66, AD
    "131_S_5138" # M, 60, AD
    
]

 

for patient_id in patients:
    process_patient(patient_id, dicom_root, output_root)
